In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd 
import numpy as np 
from datetime import datetime

In [2]:
train = pd.read_csv('./Dataset/train_dataset.csv')

In [3]:
train.shape

(10194065, 20)

In [4]:
train.head()

,SrcAddr,DstAddr,Sport,Dport,sTos,dTos,Dir,State,Proto,Dur,TotPkts,TotBytes,SrcBytes,BotnetName,Label,StartTimeHour,StartTimeMinute,StartTimeSecond,isBotnet,isSpam
0,275030,40504,303,2053634,0,0,0,117,3,0.207871,69,61300,1633,0,0,10,45,4,0,0
1,1,3393363,359,3252629,0,0,3,16,4,1003.163818,4,1977,155,0,0,10,10,17,0,0
2,529567,7,527290,44,0,0,3,16,4,0.025474,2,137,76,0,0,9,37,20,0,0
3,359,4556,224,7230,0,0,0,117,3,0.550671,13,2576,947,0,0,10,59,38,0,0
4,1,3393363,58,3252629,0,0,3,16,4,0.237174,2,235,160,0,0,9,37,29,0,0


In [5]:
X_botnet = train.drop(columns=['isBotnet', 'isSpam', 'Label', 'BotnetName'])
y_botnet = train['isBotnet']

In [6]:
X_botnet.shape

(10194065, 16)

In [11]:
X_botnet.head()

,SrcAddr,DstAddr,Sport,Dport,sTos,dTos,Dir,State,Proto,Dur,TotPkts,TotBytes,SrcBytes,StartTimeHour,StartTimeMinute,StartTimeSecond
0,275030,40504,303,2053634,0,0,0,117,3,0.207871,69,61300,1633,10,45,4
1,1,3393363,359,3252629,0,0,3,16,4,1003.163818,4,1977,155,10,10,17
2,529567,7,527290,44,0,0,3,16,4,0.025474,2,137,76,9,37,20
3,359,4556,224,7230,0,0,0,117,3,0.550671,13,2576,947,10,59,38
4,1,3393363,58,3252629,0,0,3,16,4,0.237174,2,235,160,9,37,29


In [ ]:
# selected_features_botnet = [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1] # 3 20
# selected_features_botnet = [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] # 4 10
# selected_features_botnet = [1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0] # 5 15
# SrcAddr, Dir, Proto

2 6 8 16

In [7]:
y_botnet.shape

(10194065,)

In [8]:
spam_dataset = train[train['isBotnet'] == 1]
X_spam = spam_dataset.drop(columns=['isBotnet', 'isSpam', 'Label'])
y_spam = spam_dataset['isSpam']

In [9]:
X_spam.shape

(600289, 17)

In [13]:
X_spam.head()

,SrcAddr,DstAddr,Sport,Dport,sTos,dTos,Dir,State,Proto,Dur,TotPkts,TotBytes,SrcBytes,BotnetName,StartTimeHour,StartTimeMinute,StartTimeSecond
7,55676,100,800,2053634,0,0,0,313,3,0.287393,11,1606,987,3,16,24,13
9,55676,41747,2,2053634,0,0,0,201,0,0.000000,1,1066,1066,5,14,18,46
16,55742,12749,1728,2053634,0,0,0,321,3,11.429351,9,2920,786,3,14,53,11
29,272948,8307,442,2053634,0,0,0,321,3,3.262919,11,2047,1353,3,15,36,4
39,61731,16234,16165,2053634,0,0,0,321,3,60.005222,8,1687,500,5,16,31,19


In [ ]:
# selected_features_spam = [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0] # 3 20
# selected_features_spam = [0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1] # 4 10
# selected_features_spam = [1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1] # 5 15
# Dport

In [10]:
y_spam.shape

(600289,)

# BPSO

In [14]:
# BPSO

# Sigmoid function to convert velocity into probabilities
def sigmoid(v):
    return 1 / (1 + np.exp(-v))

# Function to update position (binary)
def update_position(velocity):
    probability = sigmoid(velocity)
    new_position = np.random.rand(len(velocity)) < probability
    return new_position.astype(int)  # Convert boolean to binary (0 or 1)

# Fitness function (based on model accuracy)
def fitness_function(selected_features, X, y):
    # Ensure selected_features is a valid boolean mask
    selected_features = selected_features.astype(bool)
    
    if np.sum(selected_features) == 0:  # Avoid selecting no features
        return 1.0  # Poor fitness if no features are selected
    
    # Select features based on the boolean mask
    # If X is a DataFrame, you can use X.loc[:, selected_features]
    # If X is a NumPy array, simply use X[:, selected_features]
    if isinstance(X, np.ndarray):
        selected_X = X[:, selected_features]
    elif isinstance(X, pd.DataFrame):
        selected_X = X.loc[:, selected_features]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(selected_X, y, test_size=0.25, random_state=42)

    # Train a decision tree classifier
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)

    # Predict and calculate accuracy
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return 1 - accuracy  # Return 1 - accuracy to minimize the fitness

# BPSO Algorithm for Feature Selection
def bpso_feature_selection(X, y, n_particles, max_iter, w=0.5, c1=1.5, c2=1.5):
    n_features = X.shape[1]
    
    # Initialize particle positions (binary) and velocities
    positions = np.random.randint(2, size=(n_particles, n_features))
    velocities = np.random.uniform(-1, 1, (n_particles, n_features))
    
    # Initialize personal best positions and global best position
    pbest_positions = positions.copy()
    pbest_fitness = np.array([fitness_function(positions[i], X, y) for i in range(n_particles)])
    
    gbest_position = pbest_positions[np.argmin(pbest_fitness)]
    gbest_fitness = np.min(pbest_fitness)
    
    # BPSO main loop
    for iteration in range(max_iter):
        for i in range(n_particles):
            # Update velocity
            r1, r2 = np.random.rand(n_features), np.random.rand(n_features)
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (pbest_positions[i] - positions[i]) +
                             c2 * r2 * (gbest_position - positions[i]))
            
            # Update position (convert velocity to binary)
            positions[i] = update_position(velocities[i])
            
            # Calculate new fitness
            fitness = fitness_function(positions[i], X, y)
            
            # Update personal best if the current position is better
            if fitness < pbest_fitness[i]:
                pbest_positions[i] = positions[i]
                pbest_fitness[i] = fitness
        
        # Update global best if a better solution is found
        best_particle_idx = np.argmin(pbest_fitness)
        if pbest_fitness[best_particle_idx] < gbest_fitness:
            gbest_position = pbest_positions[best_particle_idx]
            gbest_fitness = pbest_fitness[best_particle_idx]
        
        print(f"Iteration {iteration+1}/{max_iter}, Best Fitness: {gbest_fitness}")
    
    return gbest_position, 1 - gbest_fitness  # Return best feature subset and corresponding accuracy

## Eksperimen 1

In [8]:
# DT-1
best_features, best_accuracy = bpso_feature_selection(X_botnet, y_botnet)

Iteration 1/100, Best Fitness: 0.0
Iteration 2/100, Best Fitness: 0.0
Iteration 3/100, Best Fitness: 0.0
Iteration 4/100, Best Fitness: 0.0
Iteration 5/100, Best Fitness: 0.0
Iteration 6/100, Best Fitness: 0.0
Iteration 7/100, Best Fitness: 0.0
Iteration 8/100, Best Fitness: 0.0
Iteration 9/100, Best Fitness: 0.0
Iteration 10/100, Best Fitness: 0.0
Iteration 11/100, Best Fitness: 0.0
Iteration 12/100, Best Fitness: 0.0
Iteration 13/100, Best Fitness: 0.0
Iteration 14/100, Best Fitness: 0.0
Iteration 15/100, Best Fitness: 0.0
Iteration 16/100, Best Fitness: 0.0
Iteration 17/100, Best Fitness: 0.0
Iteration 18/100, Best Fitness: 0.0
Iteration 19/100, Best Fitness: 0.0
Iteration 20/100, Best Fitness: 0.0
Iteration 21/100, Best Fitness: 0.0
Iteration 22/100, Best Fitness: 0.0
Iteration 23/100, Best Fitness: 0.0
Iteration 24/100, Best Fitness: 0.0
Iteration 25/100, Best Fitness: 0.0
Iteration 26/100, Best Fitness: 0.0
Iteration 27/100, Best Fitness: 0.0
Iteration 28/100, Best Fitness: 0.0
I

In [9]:
best_features, best_accuracy

(array([1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1]), 1.0)

In [15]:
# DT-2
best_features_spam, best_accuracy_spam = bpso_feature_selection(X_spam, y_spam)

Iteration 1/100, Best Fitness: 0.0
Iteration 2/100, Best Fitness: 0.0
Iteration 3/100, Best Fitness: 0.0
Iteration 4/100, Best Fitness: 0.0
Iteration 5/100, Best Fitness: 0.0
Iteration 6/100, Best Fitness: 0.0
Iteration 7/100, Best Fitness: 0.0
Iteration 8/100, Best Fitness: 0.0
Iteration 9/100, Best Fitness: 0.0
Iteration 10/100, Best Fitness: 0.0
Iteration 11/100, Best Fitness: 0.0
Iteration 12/100, Best Fitness: 0.0
Iteration 13/100, Best Fitness: 0.0
Iteration 14/100, Best Fitness: 0.0
Iteration 15/100, Best Fitness: 0.0
Iteration 16/100, Best Fitness: 0.0
Iteration 17/100, Best Fitness: 0.0
Iteration 18/100, Best Fitness: 0.0
Iteration 19/100, Best Fitness: 0.0
Iteration 20/100, Best Fitness: 0.0
Iteration 21/100, Best Fitness: 0.0
Iteration 22/100, Best Fitness: 0.0
Iteration 23/100, Best Fitness: 0.0
Iteration 24/100, Best Fitness: 0.0
Iteration 25/100, Best Fitness: 0.0
Iteration 26/100, Best Fitness: 0.0
Iteration 27/100, Best Fitness: 0.0
Iteration 28/100, Best Fitness: 0.0
I

In [16]:
best_features_spam, best_accuracy_spam

(array([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]),
 1.0)

## Eksperimen 2

In [17]:
# DT-1 (X, y, n_particles, max_iter, w, c1, c2) inertia, cognitive, and social components
best_features, best_accuracy = bpso_feature_selection(X_botnet, y_botnet, n_particles=30, max_iter=100, w=0.5, c1=1.5, c2=1.5)

Iteration 1/100, Best Fitness: 0.0
Iteration 2/100, Best Fitness: 0.0
Iteration 3/100, Best Fitness: 0.0
Iteration 4/100, Best Fitness: 0.0
Iteration 5/100, Best Fitness: 0.0
Iteration 6/100, Best Fitness: 0.0
Iteration 7/100, Best Fitness: 0.0
Iteration 8/100, Best Fitness: 0.0
Iteration 9/100, Best Fitness: 0.0
Iteration 10/100, Best Fitness: 0.0
Iteration 11/100, Best Fitness: 0.0
Iteration 12/100, Best Fitness: 0.0
Iteration 13/100, Best Fitness: 0.0
Iteration 14/100, Best Fitness: 0.0
Iteration 15/100, Best Fitness: 0.0
Iteration 16/100, Best Fitness: 0.0
Iteration 17/100, Best Fitness: 0.0
Iteration 18/100, Best Fitness: 0.0
Iteration 19/100, Best Fitness: 0.0
Iteration 20/100, Best Fitness: 0.0
Iteration 21/100, Best Fitness: 0.0
Iteration 22/100, Best Fitness: 0.0
Iteration 23/100, Best Fitness: 0.0
Iteration 24/100, Best Fitness: 0.0
Iteration 25/100, Best Fitness: 0.0
Iteration 26/100, Best Fitness: 0.0
Iteration 27/100, Best Fitness: 0.0
Iteration 28/100, Best Fitness: 0.0
I

In [18]:
best_features, best_accuracy

(array([1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0]), 1.0)

In [19]:
# DT-2
best_features_spam, best_accuracy_spam = bpso_feature_selection(X_spam, y_spam, n_particles=30, max_iter=100, w=0.5, c1=1.5, c2=1.5)

Iteration 1/100, Best Fitness: 0.0
Iteration 2/100, Best Fitness: 0.0
Iteration 3/100, Best Fitness: 0.0
Iteration 4/100, Best Fitness: 0.0
Iteration 5/100, Best Fitness: 0.0
Iteration 6/100, Best Fitness: 0.0
Iteration 7/100, Best Fitness: 0.0
Iteration 8/100, Best Fitness: 0.0
Iteration 9/100, Best Fitness: 0.0
Iteration 10/100, Best Fitness: 0.0
Iteration 11/100, Best Fitness: 0.0
Iteration 12/100, Best Fitness: 0.0
Iteration 13/100, Best Fitness: 0.0
Iteration 14/100, Best Fitness: 0.0
Iteration 15/100, Best Fitness: 0.0
Iteration 16/100, Best Fitness: 0.0
Iteration 17/100, Best Fitness: 0.0
Iteration 18/100, Best Fitness: 0.0
Iteration 19/100, Best Fitness: 0.0
Iteration 20/100, Best Fitness: 0.0
Iteration 21/100, Best Fitness: 0.0
Iteration 22/100, Best Fitness: 0.0
Iteration 23/100, Best Fitness: 0.0
Iteration 24/100, Best Fitness: 0.0
Iteration 25/100, Best Fitness: 0.0
Iteration 26/100, Best Fitness: 0.0
Iteration 27/100, Best Fitness: 0.0
Iteration 28/100, Best Fitness: 0.0
I

In [20]:
best_features_spam, best_accuracy_spam

(array([0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1]),
 1.0)

## Eksperimen 3 (All Sensor)

In [11]:
# DT-1 (X, y, n_particles, max_iter, w, c1, c2) inertia, cognitive, and social components
best_features, best_accuracy = bpso_feature_selection(X_botnet, y_botnet, n_particles=20, max_iter=30)

Iteration 1/30, Best Fitness: 3.923850615539237e-07
Iteration 2/30, Best Fitness: 3.923850615539237e-07
Iteration 3/30, Best Fitness: 3.923850615539237e-07
Iteration 4/30, Best Fitness: 0.0
Iteration 5/30, Best Fitness: 0.0
Iteration 6/30, Best Fitness: 0.0
Iteration 7/30, Best Fitness: 0.0
Iteration 8/30, Best Fitness: 0.0
Iteration 9/30, Best Fitness: 0.0
Iteration 10/30, Best Fitness: 0.0
Iteration 11/30, Best Fitness: 0.0
Iteration 12/30, Best Fitness: 0.0
Iteration 13/30, Best Fitness: 0.0
Iteration 14/30, Best Fitness: 0.0
Iteration 15/30, Best Fitness: 0.0
Iteration 16/30, Best Fitness: 0.0
Iteration 17/30, Best Fitness: 0.0
Iteration 18/30, Best Fitness: 0.0
Iteration 19/30, Best Fitness: 0.0
Iteration 20/30, Best Fitness: 0.0
Iteration 21/30, Best Fitness: 0.0
Iteration 22/30, Best Fitness: 0.0
Iteration 23/30, Best Fitness: 0.0
Iteration 24/30, Best Fitness: 0.0
Iteration 25/30, Best Fitness: 0.0
Iteration 26/30, Best Fitness: 0.0
Iteration 27/30, Best Fitness: 0.0
Iteration 

In [12]:
best_features, best_accuracy

(array([1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1]), 1.0)

In [13]:
# DT-2
best_features_spam, best_accuracy_spam = bpso_feature_selection(X_spam, y_spam, n_particles=20, max_iter=30)

Iteration 1/30, Best Fitness: 0.0
Iteration 2/30, Best Fitness: 0.0
Iteration 3/30, Best Fitness: 0.0
Iteration 4/30, Best Fitness: 0.0
Iteration 5/30, Best Fitness: 0.0
Iteration 6/30, Best Fitness: 0.0
Iteration 7/30, Best Fitness: 0.0
Iteration 8/30, Best Fitness: 0.0
Iteration 9/30, Best Fitness: 0.0
Iteration 10/30, Best Fitness: 0.0
Iteration 11/30, Best Fitness: 0.0
Iteration 12/30, Best Fitness: 0.0
Iteration 13/30, Best Fitness: 0.0
Iteration 14/30, Best Fitness: 0.0
Iteration 15/30, Best Fitness: 0.0
Iteration 16/30, Best Fitness: 0.0
Iteration 17/30, Best Fitness: 0.0
Iteration 18/30, Best Fitness: 0.0
Iteration 19/30, Best Fitness: 0.0
Iteration 20/30, Best Fitness: 0.0
Iteration 21/30, Best Fitness: 0.0
Iteration 22/30, Best Fitness: 0.0
Iteration 23/30, Best Fitness: 0.0
Iteration 24/30, Best Fitness: 0.0
Iteration 25/30, Best Fitness: 0.0
Iteration 26/30, Best Fitness: 0.0
Iteration 27/30, Best Fitness: 0.0
Iteration 28/30, Best Fitness: 0.0
Iteration 29/30, Best Fitness

In [14]:
best_features_spam, best_accuracy_spam

(array([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]), 1.0)

## Eksperimen 4 (10 particle)

In [15]:
# DT-1 (X, y, n_particles, max_iter, w, c1, c2) inertia, cognitive, and social components
best_features, best_accuracy = bpso_feature_selection(X_botnet, y_botnet, n_particles=10, max_iter=30)

Iteration 1/30, Best Fitness: 3.923850615539237e-07
Iteration 2/30, Best Fitness: 3.923850615539237e-07
Iteration 3/30, Best Fitness: 3.923850615539237e-07
Iteration 4/30, Best Fitness: 3.923850615539237e-07
Iteration 5/30, Best Fitness: 3.923850615539237e-07
Iteration 6/30, Best Fitness: 3.923850615539237e-07
Iteration 7/30, Best Fitness: 3.923850615539237e-07
Iteration 8/30, Best Fitness: 3.923850615539237e-07
Iteration 9/30, Best Fitness: 3.923850615539237e-07
Iteration 10/30, Best Fitness: 3.923850615539237e-07
Iteration 11/30, Best Fitness: 3.923850615539237e-07
Iteration 12/30, Best Fitness: 3.923850615539237e-07
Iteration 13/30, Best Fitness: 3.923850615539237e-07
Iteration 14/30, Best Fitness: 3.923850615539237e-07
Iteration 15/30, Best Fitness: 0.0
Iteration 16/30, Best Fitness: 0.0
Iteration 17/30, Best Fitness: 0.0
Iteration 18/30, Best Fitness: 0.0
Iteration 19/30, Best Fitness: 0.0
Iteration 20/30, Best Fitness: 0.0
Iteration 21/30, Best Fitness: 0.0
Iteration 22/30, Best 

In [16]:
best_features

array([1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [18]:
# DT-2
best_features_spam, best_accuracy_spam = bpso_feature_selection(X_spam, y_spam, n_particles=10, max_iter=30)

Iteration 1/30, Best Fitness: 0.0
Iteration 2/30, Best Fitness: 0.0
Iteration 3/30, Best Fitness: 0.0
Iteration 4/30, Best Fitness: 0.0
Iteration 5/30, Best Fitness: 0.0
Iteration 6/30, Best Fitness: 0.0
Iteration 7/30, Best Fitness: 0.0
Iteration 8/30, Best Fitness: 0.0
Iteration 9/30, Best Fitness: 0.0
Iteration 10/30, Best Fitness: 0.0
Iteration 11/30, Best Fitness: 0.0
Iteration 12/30, Best Fitness: 0.0
Iteration 13/30, Best Fitness: 0.0
Iteration 14/30, Best Fitness: 0.0
Iteration 15/30, Best Fitness: 0.0
Iteration 16/30, Best Fitness: 0.0
Iteration 17/30, Best Fitness: 0.0
Iteration 18/30, Best Fitness: 0.0
Iteration 19/30, Best Fitness: 0.0
Iteration 20/30, Best Fitness: 0.0
Iteration 21/30, Best Fitness: 0.0
Iteration 22/30, Best Fitness: 0.0
Iteration 23/30, Best Fitness: 0.0
Iteration 24/30, Best Fitness: 0.0
Iteration 25/30, Best Fitness: 0.0
Iteration 26/30, Best Fitness: 0.0
Iteration 27/30, Best Fitness: 0.0
Iteration 28/30, Best Fitness: 0.0
Iteration 29/30, Best Fitness

In [20]:
best_features_spam

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1])

## Experiment 5 (15 particle)

In [21]:
# DT-1 (X, y, n_particles, max_iter, w, c1, c2) inertia, cognitive, and social components
best_features, best_accuracy = bpso_feature_selection(X_botnet, y_botnet, n_particles=15, max_iter=30)

Iteration 1/30, Best Fitness: 3.923850615539237e-07
Iteration 2/30, Best Fitness: 3.923850615539237e-07
Iteration 3/30, Best Fitness: 0.0
Iteration 4/30, Best Fitness: 0.0
Iteration 5/30, Best Fitness: 0.0
Iteration 6/30, Best Fitness: 0.0
Iteration 7/30, Best Fitness: 0.0
Iteration 8/30, Best Fitness: 0.0
Iteration 9/30, Best Fitness: 0.0
Iteration 10/30, Best Fitness: 0.0
Iteration 11/30, Best Fitness: 0.0
Iteration 12/30, Best Fitness: 0.0
Iteration 13/30, Best Fitness: 0.0
Iteration 14/30, Best Fitness: 0.0
Iteration 15/30, Best Fitness: 0.0
Iteration 16/30, Best Fitness: 0.0
Iteration 17/30, Best Fitness: 0.0
Iteration 18/30, Best Fitness: 0.0
Iteration 19/30, Best Fitness: 0.0
Iteration 20/30, Best Fitness: 0.0
Iteration 21/30, Best Fitness: 0.0
Iteration 22/30, Best Fitness: 0.0
Iteration 23/30, Best Fitness: 0.0
Iteration 24/30, Best Fitness: 0.0
Iteration 25/30, Best Fitness: 0.0
Iteration 26/30, Best Fitness: 0.0
Iteration 27/30, Best Fitness: 0.0
Iteration 28/30, Best Fitnes

In [22]:
best_features

array([1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0])

In [23]:
# DT-2
best_features_spam, best_accuracy_spam = bpso_feature_selection(X_spam, y_spam, n_particles=15, max_iter=30)

Iteration 1/30, Best Fitness: 0.0
Iteration 2/30, Best Fitness: 0.0
Iteration 3/30, Best Fitness: 0.0
Iteration 4/30, Best Fitness: 0.0
Iteration 5/30, Best Fitness: 0.0
Iteration 6/30, Best Fitness: 0.0
Iteration 7/30, Best Fitness: 0.0
Iteration 8/30, Best Fitness: 0.0
Iteration 9/30, Best Fitness: 0.0
Iteration 10/30, Best Fitness: 0.0
Iteration 11/30, Best Fitness: 0.0
Iteration 12/30, Best Fitness: 0.0
Iteration 13/30, Best Fitness: 0.0
Iteration 14/30, Best Fitness: 0.0
Iteration 15/30, Best Fitness: 0.0
Iteration 16/30, Best Fitness: 0.0
Iteration 17/30, Best Fitness: 0.0
Iteration 18/30, Best Fitness: 0.0
Iteration 19/30, Best Fitness: 0.0
Iteration 20/30, Best Fitness: 0.0
Iteration 21/30, Best Fitness: 0.0
Iteration 22/30, Best Fitness: 0.0
Iteration 23/30, Best Fitness: 0.0
Iteration 24/30, Best Fitness: 0.0
Iteration 25/30, Best Fitness: 0.0
Iteration 26/30, Best Fitness: 0.0
Iteration 27/30, Best Fitness: 0.0
Iteration 28/30, Best Fitness: 0.0
Iteration 29/30, Best Fitness

In [24]:
best_features_spam

array([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1])